In [12]:
import sys

import tensorflow as tf
from datetime import datetime

from lib.dataset.dataset_mono import dataset_mono_predict

In [13]:
# dirpath="/home/bryan-smithl/Documents/repos/PhD-Rainfall-Radar/aimodel/output/rainfallwater_records_embed_2022-10-06_contrast_embed_umap_d512e19_tfrecord"
dirpath="/mnt/research-data/main/rainfallwater_records_tfrecord"

In [14]:
dataset = dataset_mono_predict(
	dirpath_input=dirpath,
	water_threshold=0.1,
	# shape_water_desired=[94, 94],
	parallel_reads_multiplier=1.5 # Mangles the ordering. For counting things this doesn't matter
)

DEBUG DATASET:rainfall shape [7, 174, 105] / w 105 h 174
DEBUG DATASET:water shape [348, 210]
DEBUG DATASET:water_threshold 0.1
DEBUG DATASET:water_bins 2
DEBUG DATASET:output_size 100
DEBUG DATASET:input_size 100
DEBUG DATASET:water_offset x 55 y 124
DEBUG DATASET:rainfall_offset x 3 y 37
DEBUG:dataset BEFORE_SQUEEZE water (100, 100, 1)
DEBUG:dataset AFTER_SQUEEZE water (100, 100)
DEBUG DATASET_OUT:rainfall shape (100, 100, 7)
DEBUG DATASET_OUT:water shape (100, 100)


In [15]:
i = 0
counts = tf.constant([0, 0], dtype=tf.int64)
for (items, label) in dataset:
	label = tf.cast(label, tf.int32)
	step_counts = tf.math.bincount(tf.reshape(label, [-1]))
	counts += tf.cast(step_counts, dtype=tf.int64)
	# print("STEP counts", counts, "step_counts", step_counts)
	i += 1
	if i % 100 == 0:
		sys.stderr.write(f"Processed {i} batches\r")

msg = f"Complete at {datetime.now()}. Counts:\n"+"\n".join([ str(i)+": "+str(count) for i,count in enumerate(counts.numpy().tolist()) ])
print(f"\n{msg}")



{msg}


In [18]:
total = tf.math.reduce_sum(counts)

percentages = (tf.cast(counts, tf.float64) / tf.cast(total, tf.float64)) * 100.0

msg = f"Total {total.numpy()} cells, Percentages:\n"+"\n".join(
    [str(i)+": "+str(count)+"%" for i, count in enumerate(percentages.numpy().tolist())]
)
print(f"\n{msg}")


Total 14817920000 cells, Percentages:
0: 77.92587213320088%
1: 22.074127866799117%
